<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_convnets/class_02/12%20-%20C2_W2_Lab_2_horses_v_humans_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Aumento de dados no conjunto de dados Horses or Humans

No laboratório anterior, você viu como o aumento de dados ajudou a melhorar o desempenho do modelo em dados não vistos.

Ao ajustar as imagens de treinamento de cães e gatos, o modelo conseguiu aprender recursos que também são representativos dos dados de validação.

No entanto, a aplicação do aumento de dados exige um bom entendimento do conjunto de dados.

A simples transformação aleatória nem sempre produzirá bons resultados. 

Nas próximas células, você aplicará as mesmas técnicas ao conjunto de dados `Horses or Humans` e analisará os resultados.

In [ ]:
# Faça o download do conjunto de treinamento
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

In [ ]:
# Baixar o conjunto de validação
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

In [ ]:
import os
import zipfile

# Extrair o arquivo
zip_ref = zipfile.ZipFile('./horse-or-human.zip', 'r')
zip_ref.extractall('tmp/horse-or-human')

zip_ref = zipfile.ZipFile('./validation-horse-or-human.zip', 'r')
zip_ref.extractall('tmp/validation-horse-or-human')

zip_ref.close()

# Diretório com fotos de cavalos de treinamento
train_horse_dir = os.path.join('tmp/horse-or-human/horses')

# Diretório com imagens humanas de treinamento
train_human_dir = os.path.join('tmp/horse-or-human/humans')

# Diretório com imagens de cavalos de validação
validation_horse_dir = os.path.join('tmp/validation-horse-or-human/horses')

# Diretório com imagens humanas de validação
validation_human_dir = os.path.join('tmp/validation-horse-or-human/humans')

In [ ]:
import tensorflow as tf

# Construir o modelo
model = tf.keras.models.Sequential([
    # Observe que a forma de entrada é o tamanho desejado da imagem 300x300 com 3 bytes de cor
    # Essa é a primeira convolução
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # A segunda convolução
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A terceira convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quarta convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quinta convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Achatar os resultados para alimentar um DNN
    tf.keras.layers.Flatten(),
    # Camada oculta de 512 neurônios
    tf.keras.layers.Dense(512, activation='relu'),
# Apenas 1 neurônio de saída. Ele conterá um valor de 0 a 1, sendo 0 para uma classe ("cavalos") e 1 para a outra ("humanos")
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Definir parâmetros de treinamento
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Aplicar data augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

# Fluco de imagens de treinamento em lotes de 128 usando o gerador train_datagen
train_generator = train_datagen.flow_from_directory(
        'tmp/horse-or-human/',  # Esse é o diretório de origem das imagens de treinamento
        target_size=(300, 300),   # Todas as imagens serão redimensionadas para 300x300
        batch_size=128,
        # Como usamos a perda binary_crossentropy, precisamos de rótulos binários
        class_mode='binary')

# Fluco de imagens de treinamento em lotes de 128 usando o gerador train_datagen
validation_generator = validation_datagen.flow_from_directory(
        'tmp/validation-horse-or-human/',  # Esse é o diretório de origem das imagens de validação
        target_size=(300, 300),  # Todas as imagens serão redimensionadas para 300x300
        batch_size=32,
        # Como usamos a perda binary_crossentropy, precisamos de rótulos binários
        class_mode='binary')

In [ ]:
# Constante para épocas
EPOCHS = 20

# Treinar o modelo
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=EPOCHS,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

In [ ]:
import matplotlib.pyplot as plt

# Plotar os resultados do modelo
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Acurácia de Treino')
plt.plot(epochs, val_acc, 'b', label='Acurácia de Validação')
plt.title('Acurácia de Treino e Validação')

plt.figure()

plt.plot(epochs, loss, 'r', label='Perda de Treino')
plt.plot(epochs, val_loss, 'b', label='Perda de Validação')
plt.title('Perda de Treino e Validação')
plt.legend()

plt.show()

Como você pode ver nos resultados, as técnicas de pré-processamento usadas para aumentar os dados não ajudaram muito nos resultados.

A acurácia da validação está flutuando e não está tendendo a aumentar como a acurácia do treinamento.

Isso pode ocorrer porque os dados de treinamento adicionais gerados ainda não representam os recursos nos dados de validação.

Por exemplo, algumas poses humanas ou de cavalos no conjunto de validação não podem ser imitadas pelas técnicas de processamento de imagens fornecidas pelo `ImageDataGenerator`.

Pode ser também que o plano de fundo das imagens de treinamento também seja aprendido, de modo que o plano de fundo branco do conjunto de validação esteja confundindo o modelo, mesmo com o corte.

Tente examinar as imagens de validação no diretório `tmp/validation-horse-or-human` (observação: se estiver usando o Colab, você poderá usar o explorador de arquivos à esquerda para explorar as imagens) e veja se é possível aumentar as imagens de treinamento para que correspondam às suas características.

Se isso não for possível, neste ponto você pode considerar outras técnicas, o que verá nas próximas lições.